Repost: [使用sklearn做单机特征工程](http://www.cnblogs.com/jasonfreak/p/5448385.html)

## 1. 特征工程是什么？

有这么一句话在业界广泛流传：数据和特征决定了机器学习的上限，而模型和算法只是逼近这个上限而已。那特征工程到底是什么呢？顾名思义，其本质是一项工程活动，目的是最大限度地从原始数据中提取特征以供算法和模型使用。通过总结和归纳，人们认为特征工程包括以下方面：

![](img/outline.jpg)

特征处理是特征工程的核心部分，sklearn提供了较为完整的特征处理方法，包括数据预处理，特征选择，降维等。首次接触到sklearn，通常会被其丰富且方便的算法模型库吸引，但是这里介绍的特征处理库也十分强大！

本文中使用sklearn中的IRIS（鸢尾花）数据集来对特征处理功能进行说明。IRIS数据集由Fisher在1936年整理，包含4个特征（Sepal.Length（花萼长度）、Sepal.Width（花萼宽度）、Petal.Length（花瓣长度）、Petal.Width（花瓣宽度）），特征值都为正浮点数，单位为厘米。目标值为鸢尾花的分类（Iris Setosa（山鸢尾）、Iris Versicolour（杂色鸢尾），Iris Virginica（维吉尼亚鸢尾））。导入IRIS数据集的代码如下：

In [81]:
# sklearn version: 0.18.1
from sklearn.datasets import load_iris

# 导入IRIR数据集
iris = load_iris()

In [2]:
iris.data

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.4,  3.7,  1.5,  0.2],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 4.8,  3. ,  1.4,  0.1],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.7,  4.4,  1.5,  0.4],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.1,  3.5,  1.4,  0.3],
       [ 5.7,  3.8,  1.7,  0.3],
       [ 5.1,  3.8,  1.5,  0.3],
       [ 5.4,  3.4,  1.7,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5. ,  3.4,  1.6,  0.4],
       [ 5.2,  3.5,  1.5,  0.2],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 4.7,  3.2,  1.6,  0.2],
       [ 4

In [3]:
# 目标向量
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [4]:
iris.target.shape

(150,)

## 2. 数据预处理

通过特征提取，我们能得到未经处理的特征，这时的特征可能有以下问题：

+ 不属于同一量纲：即特征的规格不一样，不能够放在一起比较。无量纲化可以解决这一问题。
+ 信息冗余：对于某些定量特征，其包含的有效信息为区间划分，例如学习成绩，假若只关心“及格”或不“及格”，那么需要将定量的考分，转换成“1”和“0”表示及格和未及格。二值化可以解决这一问题。
+ 定性特征不能直接使用：某些机器学习算法和模型只能接受定量特征的输入，那么需要将定性特征转换为定量特征。最简单的方式是为每一种定性值指定一个定量值，但是这种方式过于灵活，增加了调参的工作。通常使用哑编码的方式将定性特征转换为定量特征：假设有N种定性值，则将这一个特征扩展为N种特征，当原始特征值为第i种定性值时，第i个扩展特征赋值为1，其他扩展特征赋值为0。哑编码的方式相比直接指定的方式，不用增加调参的工作，对于线性模型来说，使用哑编码后的特征可达到非线性的效果。
+ 存在缺失值：缺失值需要补充。
+ 信息利用率低：不同的机器学习算法和模型对数据中信息的利用是不同的，之前提到在线性模型中，使用对定性特征哑编码可以达到非线性的效果。类似地，对定量变量多项式化，或者进行其他的转换，都能达到非线性的效果。

我们使用sklearn中的preproccessing库来进行数据预处理，可以覆盖以上问题的解决方案。

#### 2.1 无量纲化

无量纲化使不同规格的数据转换到同一规格。常见的无量纲化方法有标准化和区间缩放法。标准化的前提是特征值服从正态分布，标准化后，其转换成标准正态分布。区间缩放法利用了边界值信息，将特征的取值区间缩放到某个特点的范围，例如[0, 1]等。

###### 标准化

标准化需要计算特征的均值和标准差，公式表达为：

$$x'=\frac{x-\bar{X}}{S}$$

使用preproccessing库的StandardScaler类对数据进行标准化的代码如下：

In [5]:
from sklearn.preprocessing import StandardScaler

# 标准化，返回值是标准化之后的数据
StandardScaler().fit_transform(iris.data)

array([[ -9.00681170e-01,   1.03205722e+00,  -1.34127240e+00,
         -1.31297673e+00],
       [ -1.14301691e+00,  -1.24957601e-01,  -1.34127240e+00,
         -1.31297673e+00],
       [ -1.38535265e+00,   3.37848329e-01,  -1.39813811e+00,
         -1.31297673e+00],
       [ -1.50652052e+00,   1.06445364e-01,  -1.28440670e+00,
         -1.31297673e+00],
       [ -1.02184904e+00,   1.26346019e+00,  -1.34127240e+00,
         -1.31297673e+00],
       [ -5.37177559e-01,   1.95766909e+00,  -1.17067529e+00,
         -1.05003079e+00],
       [ -1.50652052e+00,   8.00654259e-01,  -1.34127240e+00,
         -1.18150376e+00],
       [ -1.02184904e+00,   8.00654259e-01,  -1.28440670e+00,
         -1.31297673e+00],
       [ -1.74885626e+00,  -3.56360566e-01,  -1.34127240e+00,
         -1.31297673e+00],
       [ -1.14301691e+00,   1.06445364e-01,  -1.28440670e+00,
         -1.44444970e+00],
       [ -5.37177559e-01,   1.49486315e+00,  -1.28440670e+00,
         -1.31297673e+00],
       [ -1.26418478e

###### 2.1.2 区间缩放法

区间缩放法的思路有多种，常见的一种为利用两个最值进行缩放，公式表达为：

$$x'=\frac{x-Min}{Max-Mix}$$

使用preproccessing库的MinMaxScaler类对数据进行区间缩放的代码如下：

In [6]:
from sklearn.preprocessing import MinMaxScaler

MinMaxScaler().fit_transform(iris.data)

array([[ 0.22222222,  0.625     ,  0.06779661,  0.04166667],
       [ 0.16666667,  0.41666667,  0.06779661,  0.04166667],
       [ 0.11111111,  0.5       ,  0.05084746,  0.04166667],
       [ 0.08333333,  0.45833333,  0.08474576,  0.04166667],
       [ 0.19444444,  0.66666667,  0.06779661,  0.04166667],
       [ 0.30555556,  0.79166667,  0.11864407,  0.125     ],
       [ 0.08333333,  0.58333333,  0.06779661,  0.08333333],
       [ 0.19444444,  0.58333333,  0.08474576,  0.04166667],
       [ 0.02777778,  0.375     ,  0.06779661,  0.04166667],
       [ 0.16666667,  0.45833333,  0.08474576,  0.        ],
       [ 0.30555556,  0.70833333,  0.08474576,  0.04166667],
       [ 0.13888889,  0.58333333,  0.10169492,  0.04166667],
       [ 0.13888889,  0.41666667,  0.06779661,  0.        ],
       [ 0.        ,  0.41666667,  0.01694915,  0.        ],
       [ 0.41666667,  0.83333333,  0.03389831,  0.04166667],
       [ 0.38888889,  1.        ,  0.08474576,  0.125     ],
       [ 0.30555556,  0.

#### 2.2 对定量特征二值化

定量特征二值化的核心在于设定一个阈值，大于阈值的赋值为1，小于等于阈值的赋值为0，公式表达如下：

$$x'=
\begin{cases}
1, & x>threshold \\
0, & x \le threshold
\end{cases}
$$

使用preproccessing库的Binarizer类对数据进行二值化的代码如下：

In [7]:
from sklearn.preprocessing import Binarizer

# 二值化，阈值设置为3，返回值为二值化后的数据
Binarizer(threshold=3).fit_transform(iris.data)

array([[ 1.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  

#### 2.3 对定性特征唖编码

由于IRIS数据集的特征皆为定量特征，故使用其目标值进行哑编码（实际上是不需要的）。使用preproccessing库的OneHotEncoder类对数据进行哑编码的代码如下：

OneHotEncoder，独热编码；

In [8]:
from sklearn.preprocessing import OneHotEncoder

OneHotEncoder().fit_transform(iris.target.reshape((-1,1)))

<150x3 sparse matrix of type '<class 'numpy.float64'>'
	with 150 stored elements in Compressed Sparse Row format>

In [9]:
# 这里-1代表，让python根据你reshape的其他维度信息，推算出-1处的取值
iris.target.reshape((-1,1))
iris.target.shape

(150,)

#### 2.4 缺失值计算

由于IRIS数据集没有缺失值，故对数据集新增一个样本，4个特征均赋值为NaN，表示数据缺失。使用preproccessing库的Imputer类对数据进行缺失值计算的代码如下：

In [10]:
from numpy import vstack, array, nan
from sklearn.preprocessing import Imputer

# 缺失值计算，返回值为计算缺失值后的数据
# missing_value为缺失值的表示形式，默认值为NaN
# 参数startegy为缺失值的填充方式，默认值为mean
Imputer().fit_transform(vstack((array([nan, nan, nan, nan]), iris.data)))

array([[ 5.84333333,  3.054     ,  3.75866667,  1.19866667],
       [ 5.1       ,  3.5       ,  1.4       ,  0.2       ],
       [ 4.9       ,  3.        ,  1.4       ,  0.2       ],
       [ 4.7       ,  3.2       ,  1.3       ,  0.2       ],
       [ 4.6       ,  3.1       ,  1.5       ,  0.2       ],
       [ 5.        ,  3.6       ,  1.4       ,  0.2       ],
       [ 5.4       ,  3.9       ,  1.7       ,  0.4       ],
       [ 4.6       ,  3.4       ,  1.4       ,  0.3       ],
       [ 5.        ,  3.4       ,  1.5       ,  0.2       ],
       [ 4.4       ,  2.9       ,  1.4       ,  0.2       ],
       [ 4.9       ,  3.1       ,  1.5       ,  0.1       ],
       [ 5.4       ,  3.7       ,  1.5       ,  0.2       ],
       [ 4.8       ,  3.4       ,  1.6       ,  0.2       ],
       [ 4.8       ,  3.        ,  1.4       ,  0.1       ],
       [ 4.3       ,  3.        ,  1.1       ,  0.1       ],
       [ 5.8       ,  4.        ,  1.2       ,  0.2       ],
       [ 5.7       ,  4.

In [11]:
vstack?

#### 2.5 数据变换

常见的数据变换有基于多项式的、基于指数函数的、基于对数函数的。4个特征，度为2的多项式转换公式如下（转换为15个特征）：

$$(x_1^\prime,x_2^\prime,x_3^\prime,x_4^\prime,x_5^\prime,x_6^\prime,x_7^\prime,x_8^\prime,x_9^\prime,x_{10}^\prime,x_{11}^\prime,x_{12}^\prime,x_{13}^\prime,x_{14}^\prime,x_{15}^\prime)= \\ (1,x_1,x_2,x_3,x_4,x_1^2,x_1 * x_2,x_1 * x_3,x_1 * x_4,x_2^2,x_2 * x_3,x_2 * x_4,x_3^2,x_3 * x_4,x_4^2)$$

使用preproccessing库的PolynomialFeatures类对数据进行多项式转换的代码如下：

In [12]:
from sklearn.preprocessing import PolynomialFeatures

# 多项式转换
# 参数degree为度，默认值为2
PolynomialFeatures().fit_transform(iris.data)

array([[  1.  ,   5.1 ,   3.5 , ...,   1.96,   0.28,   0.04],
       [  1.  ,   4.9 ,   3.  , ...,   1.96,   0.28,   0.04],
       [  1.  ,   4.7 ,   3.2 , ...,   1.69,   0.26,   0.04],
       ..., 
       [  1.  ,   6.5 ,   3.  , ...,  27.04,  10.4 ,   4.  ],
       [  1.  ,   6.2 ,   3.4 , ...,  29.16,  12.42,   5.29],
       [  1.  ,   5.9 ,   3.  , ...,  26.01,   9.18,   3.24]])

In [13]:
import sklearn
print(sklearn.__version__)

0.18.1


　基于单变元函数的数据变换可以使用一个统一的方式完成，使用preproccessing库的FunctionTransformer对数据进行对数函数转换的代码如下：

In [14]:
from numpy import log1p
from sklearn.preprocessing import FunctionTransformer

# 自定义转换函数为对数函数的数据变换
# 第一个参数是单变元函数
FunctionTransformer(log1p).fit_transform(iris.data)

array([[ 1.80828877,  1.5040774 ,  0.87546874,  0.18232156],
       [ 1.77495235,  1.38629436,  0.87546874,  0.18232156],
       [ 1.74046617,  1.43508453,  0.83290912,  0.18232156],
       [ 1.7227666 ,  1.41098697,  0.91629073,  0.18232156],
       [ 1.79175947,  1.5260563 ,  0.87546874,  0.18232156],
       [ 1.85629799,  1.58923521,  0.99325177,  0.33647224],
       [ 1.7227666 ,  1.48160454,  0.87546874,  0.26236426],
       [ 1.79175947,  1.48160454,  0.91629073,  0.18232156],
       [ 1.68639895,  1.36097655,  0.87546874,  0.18232156],
       [ 1.77495235,  1.41098697,  0.91629073,  0.09531018],
       [ 1.85629799,  1.54756251,  0.91629073,  0.18232156],
       [ 1.75785792,  1.48160454,  0.95551145,  0.18232156],
       [ 1.75785792,  1.38629436,  0.87546874,  0.09531018],
       [ 1.66770682,  1.38629436,  0.74193734,  0.09531018],
       [ 1.91692261,  1.60943791,  0.78845736,  0.18232156],
       [ 1.90210753,  1.68639895,  0.91629073,  0.33647224],
       [ 1.85629799,  1.

#### 2.6 回顾

| 类 | 功能 | 说明 |
| :--- | :----: | ----: |
|StandardScaler	|无量纲化|	标准化，基于特征矩阵的列，将特征值转换至服从标准正态分布|
|MinMaxScaler	|无量纲化|	区间缩放，基于最大最小值，将特征值转换到[0, 1]区间上|
|Normalizer	|归一化|	基于特征矩阵的行，将样本向量转换为“单位向量”|
|Binarizer	|二值化|	基于给定阈值，将定量特征按阈值划分|
|OneHotEncoder	|哑编码|	将定性数据编码为定量数据|
|Imputer	|缺失值计算|	计算缺失值，缺失值可填充为均值等|
|PolynomialFeatures	|多项式数据转换|	多项式数据转换|
|FunctionTransformer	|自定义单元数据转换|	使用单变元的函数来转换数据|

## 3. 特征选择

当数据预处理完成后，我们需要选择有意义的特征输入机器学习的算法和模型进行训练。通常来说，从两个方面考虑来选择特征：

+ 特征是否发散：如果一个特征不发散，例如方差接近于0，也就是说样本在这个特征上基本上没有差异，这个特征对于样本的区分并没有什么用。
+ 特征与目标的相关性：这点比较显见，与目标相关性高的特征，应当优选选择。除方差法外，本文介绍的其他方法均从**相关性**考虑。

根据特征选择的形式又可以将特征选择方法分为3种：

+ Filter：过滤法，按照发散性或者相关性对各个特征进行评分，设定阈值或者待选择阈值的个数，选择特征。
+ Wrapper：包装法，根据目标函数（通常是预测效果评分），每次选择若干特征，或者排除若干特征。
+ Embedded：嵌入法，先使用某些机器学习的算法和模型进行训练，得到各个特征的权值系数，根据系数从大到小选择特征。类似于Filter方法，但是是通过训练来确定特征的优劣。

我们使用sklearn中的feature_selection库来进行特征选择。

#### 3.1 Filter

###### 3.1.1 方差选择法

使用方差选择法，先要计算各个特征的方差，然后根据阈值，选择方差大于阈值的特征。使用feature_selection库的VarianceThreshold类来选择特征的代码如下：

In [17]:
from sklearn.feature_selection import VarianceThreshold

# 方差选择法，返回值为特征选择后的数据
# 参数threshold为方差的阈值
VarianceThreshold(threshold=3).fit_transform(iris.data)

array([[ 1.4],
       [ 1.4],
       [ 1.3],
       [ 1.5],
       [ 1.4],
       [ 1.7],
       [ 1.4],
       [ 1.5],
       [ 1.4],
       [ 1.5],
       [ 1.5],
       [ 1.6],
       [ 1.4],
       [ 1.1],
       [ 1.2],
       [ 1.5],
       [ 1.3],
       [ 1.4],
       [ 1.7],
       [ 1.5],
       [ 1.7],
       [ 1.5],
       [ 1. ],
       [ 1.7],
       [ 1.9],
       [ 1.6],
       [ 1.6],
       [ 1.5],
       [ 1.4],
       [ 1.6],
       [ 1.6],
       [ 1.5],
       [ 1.5],
       [ 1.4],
       [ 1.5],
       [ 1.2],
       [ 1.3],
       [ 1.5],
       [ 1.3],
       [ 1.5],
       [ 1.3],
       [ 1.3],
       [ 1.3],
       [ 1.6],
       [ 1.9],
       [ 1.4],
       [ 1.6],
       [ 1.4],
       [ 1.5],
       [ 1.4],
       [ 4.7],
       [ 4.5],
       [ 4.9],
       [ 4. ],
       [ 4.6],
       [ 4.5],
       [ 4.7],
       [ 3.3],
       [ 4.6],
       [ 3.9],
       [ 3.5],
       [ 4.2],
       [ 4. ],
       [ 4.7],
       [ 3.6],
       [ 4.4],
       [ 4

###### 3.1.2 相关系数法

使用相关系数法，先要计算各个特征对目标值的相关系数以及相关系数的P值。用feature_selection库的SelectKBest类结合相关系数来选择特征的代码如下：

In [57]:
from sklearn.feature_selection import SelectKBest
from scipy.stats import pearsonr

# 选择K个最好的特征，返回特征选择后的数据
# 第一个参数为计算评估特征是否好的函数，该函数输入特征矩阵和目标向量，输出二元组（评分，P值）的数组，数组第i项为第i个特征的评分和P值。在此定义为计算相关系数
# 参数k为选择的特征个数
SelectKBest(lambda X, Y: (array(list(map(lambda x:pearsonr(x, Y), X.T))).T[0], array(list(map(lambda x:pearsonr(x, Y), X.T))).T[1]), k=2).fit_transform(iris.data, iris.target)

array([[ 1.4,  0.2],
       [ 1.4,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.7,  0.4],
       [ 1.4,  0.3],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.5,  0.1],
       [ 1.5,  0.2],
       [ 1.6,  0.2],
       [ 1.4,  0.1],
       [ 1.1,  0.1],
       [ 1.2,  0.2],
       [ 1.5,  0.4],
       [ 1.3,  0.4],
       [ 1.4,  0.3],
       [ 1.7,  0.3],
       [ 1.5,  0.3],
       [ 1.7,  0.2],
       [ 1.5,  0.4],
       [ 1. ,  0.2],
       [ 1.7,  0.5],
       [ 1.9,  0.2],
       [ 1.6,  0.2],
       [ 1.6,  0.4],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.6,  0.2],
       [ 1.6,  0.2],
       [ 1.5,  0.4],
       [ 1.5,  0.1],
       [ 1.4,  0.2],
       [ 1.5,  0.1],
       [ 1.2,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.1],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.3,  0.3],
       [ 1.3,  0.3],
       [ 1.3,  0.2],
       [ 1.6,  0.6],
       [ 1.9,  0.4],
       [ 1.4,  0.3],
       [ 1.6,  0.2],
       [ 1.4,

In [69]:
from sklearn.feature_selection import SelectKBest
from scipy.stats import pearsonr

# 选择K个最好的特征，返回特征选择后的数据
# 第一个参数为计算评估特征是否好的函数，该函数输入特征矩阵和目标向量，输出二元组（评分，P值）的数组，数组第i项为第i个特征的评分和P值。在此定义为计算相关系数
# 参数k为选择的特征个数
SelectKBest(lambda X, Y: tuple(map(tuple,array(list(map(lambda x:pearsonr(x, Y), X.T))).T)), k=2).fit_transform(iris.data, iris.target)

array([[ 1.4,  0.2],
       [ 1.4,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.7,  0.4],
       [ 1.4,  0.3],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.5,  0.1],
       [ 1.5,  0.2],
       [ 1.6,  0.2],
       [ 1.4,  0.1],
       [ 1.1,  0.1],
       [ 1.2,  0.2],
       [ 1.5,  0.4],
       [ 1.3,  0.4],
       [ 1.4,  0.3],
       [ 1.7,  0.3],
       [ 1.5,  0.3],
       [ 1.7,  0.2],
       [ 1.5,  0.4],
       [ 1. ,  0.2],
       [ 1.7,  0.5],
       [ 1.9,  0.2],
       [ 1.6,  0.2],
       [ 1.6,  0.4],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.6,  0.2],
       [ 1.6,  0.2],
       [ 1.5,  0.4],
       [ 1.5,  0.1],
       [ 1.4,  0.2],
       [ 1.5,  0.1],
       [ 1.2,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.1],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.3,  0.3],
       [ 1.3,  0.3],
       [ 1.3,  0.2],
       [ 1.6,  0.6],
       [ 1.9,  0.4],
       [ 1.4,  0.3],
       [ 1.6,  0.2],
       [ 1.4,

In [72]:
array(list(map(lambda x:pearsonr(x, iris.target), iris.data.T))).T # shape (2,4)

array([[  7.82561232e-01,  -4.19446200e-01,   9.49042545e-01,
          9.56463824e-01],
       [  2.89047835e-32,   9.15998497e-08,   4.15547758e-76,
          4.77500237e-81]])

In [68]:
a = array(list(map(lambda x:pearsonr(x, iris.target), iris.data.T))).T
(a[0], a[1])
array(tuple(map(tuple, a)))

array([[  7.82561232e-01,  -4.19446200e-01,   9.49042545e-01,
          9.56463824e-01],
       [  2.89047835e-32,   9.15998497e-08,   4.15547758e-76,
          4.77500237e-81]])

In [62]:
chi2(iris.data, iris.target)

(array([  10.81782088,    3.59449902,  116.16984746,   67.24482759]),
 array([  4.47651499e-03,   1.65754167e-01,   5.94344354e-26,
          2.50017968e-15]))

###### 3.1.3 卡方检验

经典的卡方检验是检验定性自变量对定性因变量的相关性。假设自变量有N种取值，因变量有M种取值，考虑自变量等于i且因变量等于j的样本频数的观察值与期望的差距，构建统计量：

$$x^2 = \sum{\frac{(A-E)^2}{E}}$$

这个统计量的含义简而言之就是自变量对因变量的相关性。用feature_selection库的SelectKBest类结合卡方检验来选择特征的代码如下：

In [24]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# 选择K个最好的特征，返回选择特征后的数据
SelectKBest(chi2, k=2).fit_transform(iris.data, iris.target)

array([[ 1.4,  0.2],
       [ 1.4,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.7,  0.4],
       [ 1.4,  0.3],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.5,  0.1],
       [ 1.5,  0.2],
       [ 1.6,  0.2],
       [ 1.4,  0.1],
       [ 1.1,  0.1],
       [ 1.2,  0.2],
       [ 1.5,  0.4],
       [ 1.3,  0.4],
       [ 1.4,  0.3],
       [ 1.7,  0.3],
       [ 1.5,  0.3],
       [ 1.7,  0.2],
       [ 1.5,  0.4],
       [ 1. ,  0.2],
       [ 1.7,  0.5],
       [ 1.9,  0.2],
       [ 1.6,  0.2],
       [ 1.6,  0.4],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.6,  0.2],
       [ 1.6,  0.2],
       [ 1.5,  0.4],
       [ 1.5,  0.1],
       [ 1.4,  0.2],
       [ 1.5,  0.1],
       [ 1.2,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.1],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.3,  0.3],
       [ 1.3,  0.3],
       [ 1.3,  0.2],
       [ 1.6,  0.6],
       [ 1.9,  0.4],
       [ 1.4,  0.3],
       [ 1.6,  0.2],
       [ 1.4,

###### 3.1.4 互信息法

经典的互信息也是评价定性自变量对定性因变量的相关性的，互信息计算公式如下：

$$I(X;Y)=\sum_{x \in X}\sum_{y \in Y}p(x,y)log\frac{p(x,y)}{p(x)p(y)}$$

为了处理定量数据，最大信息系数法被提出，使用feature_selection库的SelectKBest类结合最大信息系数法来选择特征的代码如下：

In [ ]:
from sklearn.feature_selection import SelectKBest
from minepy import MINE

# 由于MINE的设计不是函数式的，定义mic方法将其为函数式的，返回一个二元组，二元组的第2项设置成固定的P值0.5
def mic(x, y):
    m = MINE()
    m.compute_score(x, y)
    return (m.mic(), 0.5)

#选择K个最好的特征，返回特征选择后的数据
SelectKBest(lambda X, Y: array(map(lambda x:mic(x, Y), X.T)).T, k=2).fit_transform(iris.data, iris.target)

#### 3.2 Wrapper

###### 3.2.1 递归特征消除法

递归消除特征法使用一个基模型来进行多轮训练，每轮训练后，消除若干权值系数的特征，再基于新的特征集进行下一轮训练。使用feature_selection库的RFE类来选择特征的代码如下：

In [73]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# 递归特征消除法，返回特征选择后的数据
# esitmator为基模型
# 参数n_features_to_select为选择的特征个数
RFE(estimator=LogisticRegression(), n_features_to_select=2).fit_transform(iris.data, iris.target)

array([[ 3.5,  0.2],
       [ 3. ,  0.2],
       [ 3.2,  0.2],
       [ 3.1,  0.2],
       [ 3.6,  0.2],
       [ 3.9,  0.4],
       [ 3.4,  0.3],
       [ 3.4,  0.2],
       [ 2.9,  0.2],
       [ 3.1,  0.1],
       [ 3.7,  0.2],
       [ 3.4,  0.2],
       [ 3. ,  0.1],
       [ 3. ,  0.1],
       [ 4. ,  0.2],
       [ 4.4,  0.4],
       [ 3.9,  0.4],
       [ 3.5,  0.3],
       [ 3.8,  0.3],
       [ 3.8,  0.3],
       [ 3.4,  0.2],
       [ 3.7,  0.4],
       [ 3.6,  0.2],
       [ 3.3,  0.5],
       [ 3.4,  0.2],
       [ 3. ,  0.2],
       [ 3.4,  0.4],
       [ 3.5,  0.2],
       [ 3.4,  0.2],
       [ 3.2,  0.2],
       [ 3.1,  0.2],
       [ 3.4,  0.4],
       [ 4.1,  0.1],
       [ 4.2,  0.2],
       [ 3.1,  0.1],
       [ 3.2,  0.2],
       [ 3.5,  0.2],
       [ 3.1,  0.1],
       [ 3. ,  0.2],
       [ 3.4,  0.2],
       [ 3.5,  0.3],
       [ 2.3,  0.3],
       [ 3.2,  0.2],
       [ 3.5,  0.6],
       [ 3.8,  0.4],
       [ 3. ,  0.3],
       [ 3.8,  0.2],
       [ 3.2,

#### 3.3 Embedded

###### 3.3.1 基于惩罚项的特征选择方法

使用带惩罚项的基模型，除了筛选出特征外~~，同时也进行了降维~~。使用feature_selection库的SelectFromModel类结合带L1惩罚项的逻辑回归模型，来选择特征的代码如下：

In [74]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

# 带L1惩罚项的逻辑回归作为基模型的特征选择
SelectFromModel(LogisticRegression(penalty='l1', C=0.1)).fit_transform(iris.data, iris.target)

array([[ 5.1,  3.5,  1.4],
       [ 4.9,  3. ,  1.4],
       [ 4.7,  3.2,  1.3],
       [ 4.6,  3.1,  1.5],
       [ 5. ,  3.6,  1.4],
       [ 5.4,  3.9,  1.7],
       [ 4.6,  3.4,  1.4],
       [ 5. ,  3.4,  1.5],
       [ 4.4,  2.9,  1.4],
       [ 4.9,  3.1,  1.5],
       [ 5.4,  3.7,  1.5],
       [ 4.8,  3.4,  1.6],
       [ 4.8,  3. ,  1.4],
       [ 4.3,  3. ,  1.1],
       [ 5.8,  4. ,  1.2],
       [ 5.7,  4.4,  1.5],
       [ 5.4,  3.9,  1.3],
       [ 5.1,  3.5,  1.4],
       [ 5.7,  3.8,  1.7],
       [ 5.1,  3.8,  1.5],
       [ 5.4,  3.4,  1.7],
       [ 5.1,  3.7,  1.5],
       [ 4.6,  3.6,  1. ],
       [ 5.1,  3.3,  1.7],
       [ 4.8,  3.4,  1.9],
       [ 5. ,  3. ,  1.6],
       [ 5. ,  3.4,  1.6],
       [ 5.2,  3.5,  1.5],
       [ 5.2,  3.4,  1.4],
       [ 4.7,  3.2,  1.6],
       [ 4.8,  3.1,  1.6],
       [ 5.4,  3.4,  1.5],
       [ 5.2,  4.1,  1.5],
       [ 5.5,  4.2,  1.4],
       [ 4.9,  3.1,  1.5],
       [ 5. ,  3.2,  1.2],
       [ 5.5,  3.5,  1.3],
 

[L1惩罚项降维的原理在于保留多个对目标值具有同等相关性的特征中的一个](http://www.zhihu.com/question/28641663/answer/41653367)，所以没选到的特征不代表不重要。故，可结合L2惩罚项来优化。具体操作为：若一个特征在L1中的权值为1，选择在L2中权值差别不大且在L1中权值为0的特征构成同类集合，将这一集合中的特征平分L1中的权值，故需要构建一个新的逻辑回归模型：

In [75]:
from sklearn.linear_model import LogisticRegression

class LR(LogisticRegression):
    def __init__(self, threshold=0.01, dual=False, tol=1e-4, C=1.0,
                    fit_intercept=True, intercept_scaling=1, class_weight=None,
                    random_state=None, solver='liblinear', max_iter=100,
                    multi_class='ovr', verbose=0, warm_start=False, n_jobs=1):
        # 权值相近的阈值
        self.threshold = threshold
        LogisticRegression.__init__(self, penalty='l1', dual=dual, tol=tol, C=C,
                                    fit_intercept=fit_intercept, intercept_scaling=intercept_scaling, class_weight=class_weight,
                                    random_state=random_state, solver=solver, max_iter=max_iter,
                                    multi_class=multi_class, verbose=verbose, warm_start=warm_start, n_jobs=n_jobs)
        #使用同样的参数创建L2逻辑回归)
        self.l2 = LogisticRegression(penalty='l2', dual=dual, tol=tol, C=C, fit_intercept=fit_intercept, intercept_scaling=intercept_scaling,
                                     class_weight = class_weight, random_state=random_state, solver=solver, max_iter=max_iter, multi_class=multi_class, verbose=verbose, warm_start=warm_start, n_jobs=n_jobs)
    
        def fit(self, X, y, sample_weight=None):
            # 训练L1逻辑回归
            super(LR, self).fit(X, y, sample_weight=sample_weight)
            self.coef_old_ = self.coef_.copy()
            # 训练L2逻辑回归
            self.l2.fit(X, y, sample_weight=sample_weight)
            
            cntOfRow, cntOfCol = self.coef_.shape
            # 权值系数矩阵的行数对应目标值的种类数目
            for i in range(cntOfRow):
                for j in range(cntOfCol):
                    coef = self.coef_[i][j]
                    # L1逻辑回归的权值系数不为0
                    if coef != 0:
                        idx = [j]
                        # 对应再L2逻辑回归中的权值系数
                        coef1 = self.l2.coef_[i][j]
                        for k in range(cntOfCol):
                            coef2 = self.l2.coef_[i][k]
                            # 在L2逻辑回归中，权值系数之差小于设定的阈值，且在L1中对应的权值为0
                            if abs(coef1-coef2) < self.threshold and j != k and self.coef_[i][k] == 0:
                                idx.append(k)
                        # 计算这一类特征的权值系数权值
                        mean = coef / len(idx)
                        self.coef_[i][idx] = mean
            return self

使用feature_selection库的SelectFromModel类结合带L1以及L2惩罚项的逻辑回归模型，来选择特征的代码如下：

In [76]:
from sklearn.feature_selection import SelectFromModel

#带L1和L2惩罚项的逻辑回归作为基模型的特征选择
#参数threshold为权值系数之差的阈值
SelectFromModel(LR(threshold=0.5, C=0.1)).fit_transform(iris.data, iris.target)

array([[ 5.1,  3.5,  1.4],
       [ 4.9,  3. ,  1.4],
       [ 4.7,  3.2,  1.3],
       [ 4.6,  3.1,  1.5],
       [ 5. ,  3.6,  1.4],
       [ 5.4,  3.9,  1.7],
       [ 4.6,  3.4,  1.4],
       [ 5. ,  3.4,  1.5],
       [ 4.4,  2.9,  1.4],
       [ 4.9,  3.1,  1.5],
       [ 5.4,  3.7,  1.5],
       [ 4.8,  3.4,  1.6],
       [ 4.8,  3. ,  1.4],
       [ 4.3,  3. ,  1.1],
       [ 5.8,  4. ,  1.2],
       [ 5.7,  4.4,  1.5],
       [ 5.4,  3.9,  1.3],
       [ 5.1,  3.5,  1.4],
       [ 5.7,  3.8,  1.7],
       [ 5.1,  3.8,  1.5],
       [ 5.4,  3.4,  1.7],
       [ 5.1,  3.7,  1.5],
       [ 4.6,  3.6,  1. ],
       [ 5.1,  3.3,  1.7],
       [ 4.8,  3.4,  1.9],
       [ 5. ,  3. ,  1.6],
       [ 5. ,  3.4,  1.6],
       [ 5.2,  3.5,  1.5],
       [ 5.2,  3.4,  1.4],
       [ 4.7,  3.2,  1.6],
       [ 4.8,  3.1,  1.6],
       [ 5.4,  3.4,  1.5],
       [ 5.2,  4.1,  1.5],
       [ 5.5,  4.2,  1.4],
       [ 4.9,  3.1,  1.5],
       [ 5. ,  3.2,  1.2],
       [ 5.5,  3.5,  1.3],
 

###### 3.3.2 基于树模型的特征选择算法

树模型中GBDT也可用来作为基模型进行特征选择，使用feature_selection库的SelectFromModel类结合GBDT模型，来选择特征的代码如下：

In [77]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier

# GBDT作为基模型的特征选择
SelectFromModel(GradientBoostingClassifier()).fit_transform(iris.data, iris.target)

array([[ 1.4,  0.2],
       [ 1.4,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.7,  0.4],
       [ 1.4,  0.3],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.5,  0.1],
       [ 1.5,  0.2],
       [ 1.6,  0.2],
       [ 1.4,  0.1],
       [ 1.1,  0.1],
       [ 1.2,  0.2],
       [ 1.5,  0.4],
       [ 1.3,  0.4],
       [ 1.4,  0.3],
       [ 1.7,  0.3],
       [ 1.5,  0.3],
       [ 1.7,  0.2],
       [ 1.5,  0.4],
       [ 1. ,  0.2],
       [ 1.7,  0.5],
       [ 1.9,  0.2],
       [ 1.6,  0.2],
       [ 1.6,  0.4],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.6,  0.2],
       [ 1.6,  0.2],
       [ 1.5,  0.4],
       [ 1.5,  0.1],
       [ 1.4,  0.2],
       [ 1.5,  0.1],
       [ 1.2,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.1],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.3,  0.3],
       [ 1.3,  0.3],
       [ 1.3,  0.2],
       [ 1.6,  0.6],
       [ 1.9,  0.4],
       [ 1.4,  0.3],
       [ 1.6,  0.2],
       [ 1.4,

#### 3.4回顾

|类	|所属方式|	说明|
|:---|:---:|---:|
|VarianceThreshold |Filter|	方差选择法|
|SelectKBest	|Filter|	可选关联系数、卡方校验、最大信息系数作为得分计算的方法|
|RFE	|Wrapper|	递归地训练基模型，将权值系数较小的特征从特征集合中消除|
|SelectFromModel	|Embedded|	训练基模型，选择权值系数较高的特征|

## 4. 降维

当特征选择完成后，可以直接训练模型了，但是可能由于特征矩阵过大，导致计算量大，训练时间长的问题，因此降低特征矩阵维度也是必不可少的。常见的降维方法除了以上提到的基于L1惩罚项的模型以外，另外还有主成分分析法（PCA）和线性判别分析（LDA），线性判别分析本身也是一个分类模型。PCA和LDA有很多的相似点，其本质是要将原始的样本映射到维度更低的样本空间中，但是PCA和LDA的映射目标不一样：PCA是为了让映射后的样本具有最大的发散性；而LDA是为了让映射后的样本有最好的分类性能。所以说PCA是一种无监督的降维方法，而LDA是一种有监督的降维方法。

注：PCA说发散性，应该是保留更多的信息；

## 4.1 主成分分析法（PCA）

使用decomposition库的PCA类选择特征的代码如下：

In [78]:
from sklearn.decomposition import PCA

# 主成分分析法，返回降维之后的数据
# 参数n_components为主成分的数目
PCA(n_components=2).fit_transform(iris.data)

array([[-2.68420713,  0.32660731],
       [-2.71539062, -0.16955685],
       [-2.88981954, -0.13734561],
       [-2.7464372 , -0.31112432],
       [-2.72859298,  0.33392456],
       [-2.27989736,  0.74778271],
       [-2.82089068, -0.08210451],
       [-2.62648199,  0.17040535],
       [-2.88795857, -0.57079803],
       [-2.67384469, -0.1066917 ],
       [-2.50652679,  0.65193501],
       [-2.61314272,  0.02152063],
       [-2.78743398, -0.22774019],
       [-3.22520045, -0.50327991],
       [-2.64354322,  1.1861949 ],
       [-2.38386932,  1.34475434],
       [-2.6225262 ,  0.81808967],
       [-2.64832273,  0.31913667],
       [-2.19907796,  0.87924409],
       [-2.58734619,  0.52047364],
       [-2.3105317 ,  0.39786782],
       [-2.54323491,  0.44003175],
       [-3.21585769,  0.14161557],
       [-2.30312854,  0.10552268],
       [-2.35617109, -0.03120959],
       [-2.50791723, -0.13905634],
       [-2.469056  ,  0.13788731],
       [-2.56239095,  0.37468456],
       [-2.63982127,

## 线性判别分析法（LDA）

使用lda库的LDA类选择特征的代码如下：

In [79]:
from sklearn.lda import LDA

# 线性判别分析法，返回降维后的数据
# 参数n_components为降维后的数据
LDA(n_components=2).fit_transform(iris.data, iris.target)

C:\Anaconda3\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


array([[-8.0849532 ,  0.32845422],
       [-7.1471629 , -0.75547326],
       [-7.51137789, -0.23807832],
       [-6.83767561, -0.64288476],
       [-8.15781367,  0.54063935],
       [-7.72363087,  1.48232345],
       [-7.23514662,  0.3771537 ],
       [-7.62974497,  0.01667246],
       [-6.58274132, -0.98737424],
       [-7.36884116, -0.91362729],
       [-8.42181434,  0.67622968],
       [-7.24739721, -0.08292417],
       [-7.35062105, -1.0393597 ],
       [-7.59646896, -0.77671553],
       [-9.86936588,  1.61486093],
       [-9.18033614,  2.75558626],
       [-8.59760709,  1.85442217],
       [-7.7995682 ,  0.60905468],
       [-8.1000091 ,  0.99610981],
       [-8.04543611,  1.16244332],
       [-7.52046427, -0.156233  ],
       [-7.60526378,  1.22757267],
       [-8.70408249,  0.89959416],
       [-6.26374139,  0.46023935],
       [-6.59191505, -0.36199821],
       [-6.79210164, -0.93823664],
       [-6.84048091,  0.4848487 ],
       [-7.948386  ,  0.23871551],
       [-8.01209273,

In [80]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 线性判别分析法，返回降维后的数据
# 参数n_components为降维后的数据
LinearDiscriminantAnalysis(n_components=2).fit_transform(iris.data, iris.target)

array([[-8.0849532 ,  0.32845422],
       [-7.1471629 , -0.75547326],
       [-7.51137789, -0.23807832],
       [-6.83767561, -0.64288476],
       [-8.15781367,  0.54063935],
       [-7.72363087,  1.48232345],
       [-7.23514662,  0.3771537 ],
       [-7.62974497,  0.01667246],
       [-6.58274132, -0.98737424],
       [-7.36884116, -0.91362729],
       [-8.42181434,  0.67622968],
       [-7.24739721, -0.08292417],
       [-7.35062105, -1.0393597 ],
       [-7.59646896, -0.77671553],
       [-9.86936588,  1.61486093],
       [-9.18033614,  2.75558626],
       [-8.59760709,  1.85442217],
       [-7.7995682 ,  0.60905468],
       [-8.1000091 ,  0.99610981],
       [-8.04543611,  1.16244332],
       [-7.52046427, -0.156233  ],
       [-7.60526378,  1.22757267],
       [-8.70408249,  0.89959416],
       [-6.26374139,  0.46023935],
       [-6.59191505, -0.36199821],
       [-6.79210164, -0.93823664],
       [-6.84048091,  0.4848487 ],
       [-7.948386  ,  0.23871551],
       [-8.01209273,

#### 4.3 回顾

|库|类|说明|
|:---|:---:|---:|
|decomposition|PCA|主成分分析法|
|discriminant_analysis|LinearDiscriminantAnalysis|线性判别分析法|

## 5. 总结

再让我们回归一下本文开始的特征工程的思维导图，我们可以使用sklearn完成几乎所有特征处理的工作，而且不管是数据预处理，还是特征选择，抑或降维，它们都是通过某个类的方法fit_transform完成的，fit_transform要不只带一个参数：特征矩阵，要不带两个参数：特征矩阵加目标向量。这些难道都是巧合吗？还是故意设计成这样？方法fit_transform中有fit这一单词，它和训练模型的fit方法有关联吗？接下来，我将在[《使用sklearn优雅地进行数据挖掘》](http://www.cnblogs.com/jasonfreak/p/5448462.html)中阐述其中的奥妙！再让我们回归一下本文开始的特征工程的思维导图，我们可以使用sklearn完成几乎所有特征处理的工作，而且不管是数据预处理，还是特征选择，抑或降维，它们都是通过某个类的方法fit_transform完成的，fit_transform要不只带一个参数：特征矩阵，要不带两个参数：特征矩阵加目标向量。这些难道都是巧合吗？还是故意设计成这样？方法fit_transform中有fit这一单词，它和训练模型的fit方法有关联吗？接下来，我将在《使用sklearn优雅地进行数据挖掘》中阐述其中的奥妙！

应该是训练集和测试集...

## 6. 参考资料

1. [FAQ: What is dummy coding?](http://www.ats.ucla.edu/stat/mult_pkg/faq/general/dummy.htm)
2. [IRIS（鸢尾花）数据集](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris)
3. [卡方检验](http://wiki.mbalib.com/wiki/%E5%8D%A1%E6%96%B9%E6%A3%80%E9%AA%8C)
4. [干货：结合Scikit-learn介绍几种常用的特征选择方法](http://dataunion.org/14072.html)
5. [机器学习中，有哪些特征选择的工程方法？](http://www.zhihu.com/question/28641663/answer/41653367)
6. [机器学习中的数学(4)-线性判别分析（LDA）, 主成分分析(PCA)](http://www.cnblogs.com/LeftNotEasy/archive/2011/01/08/lda-and-pca-machine-learning.html)